In [ ]:
import numpy as np
from flask import Flask,request,render_template
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler

app = Flask(__name__)

@app.route("/home/")
def home():
    return render_template("home.html",img_path = "/static/img/obesity.png",text = "Click buttons to show charts")

@app.route("/dist/")
def dist():
    return render_template("home.html", img_path = "/static/img/distribution.png",text = "Demographics - distribution histograms")

@app.route("/pie/")
def pie():
    return render_template("home.html", img_path = "/static/img/pies.png",text = "Demographics - Pie charts")

@app.route("/gender/")
def gender():
    return render_template("home.html", img_path = "/static/img/gender.png",text = "The relationship between obesity and gender")

@app.route("/alochol/")
def alochol():
    return render_template("home.html", img_path = "/static/img/alcohol.png",text = "The relationship between obesity and alcohol")

@app.route("/heatmap/")
def heatmap():
    return render_template("home.html", img_path = "/static/img/heatmap.png",text = "The heatmap of variables")

@app.route("/scatter/")
def scatter():
    return render_template("home.html", img_path = "/static/img/scatter.png",text = "Data display after dimensionality reduction")

@app.route("/to_predict/")
def to_predict():
    return render_template("predict.html")

@app.route("/predict/")
def predict():
    age = request.args.get("age")
    sex = request.args.get("gender")
    height = request.args.get("height")
    weight = request.args.get("weight")
    food = request.args.get("food")
    smoke = request.args.get("smoke")
    sports = request.args.get("sports")
    datasets = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
    data = datasets[["Gender","Age","Height","Weight","FAVC","SMOKE","FAF"]]
    labels = datasets["NObeyesdad"]
    labels_ = np.unique(labels)
    label_enc = LabelEncoder()
    labels = label_enc.fit_transform(labels)
    data["Gender"] = data["Gender"].apply(lambda x:0 if x == "Male" else 1)
    data["FAVC"] = data["FAVC"].apply(lambda x:0 if x == "no" else 1)
    data["SMOKE"] = data["SMOKE"].apply(lambda x:0 if x == "no" else 1)
    stand_scaler = StandardScaler()
    stand_scaler.fit(data)
    data = stand_scaler.transform(data)
    model = KNeighborsClassifier(n_neighbors=10)
    model.fit(data,labels)
    datasets_test = np.array([[sex,age,height,weight,food,smoke,sports]])
    datasets_test = stand_scaler.transform(datasets_test)
    pred = model.predict(datasets_test)
    print("--->",labels_[pred[0]])
    return render_template("predict.html",pred = labels_[pred],img_path = "/static/img/tree.png")


# if __name__ == '__main__':
app.run()
